In [ ]:
# @title 1. 環境構築とライブラリのインストール
# 必要なライブラリをインストールします
!pip install numpy scipy librosa soundfile pydub moviepy

import os
import sys
import numpy as np
import librosa
import soundfile as sf
import urllib.parse
import glob
import subprocess
from scipy.interpolate import interp1d

print("ライブラリのインストールが完了しました。")

In [ ]:
# @title 2. 素材データのダウンロード
# モノラル音源の映画とMRSSpeechの位置情報データ(NPYデータ)をダウンロードします

# 1. 白黒映画『素晴らしき哉、人生!』（1946年）のダウンロード
movie_url = "https://dn720303.ca.archive.org/0/items/its-a-wonderful-life-1946_202108/Its-a-wonderful-life_1946.mp4"
movie_filename = "Its-a-wonderful-life_1946.mp4"

if not os.path.exists(movie_filename):
    print(f"映画をダウンロード中...: {movie_filename}")
    !wget -O "$movie_filename" "$movie_url"
else:
    print(f"ファイルは既に存在します: {movie_filename}")

# 2. MRSSpeechの位置情報データ(NPY)のダウンロード
npy_url = "https://huggingface.co/datasets/verstar/MRSAudio/resolve/main/MRSSpeech/%E3%80%8A%E7%94%9F%E6%B4%BB%E6%98%AF%E7%BE%8E%E5%A5%BD%E7%9A%84%E3%80%8B%E7%94%B5%E5%BD%B1%E5%89%A7%E6%9C%AC_1.2/%E3%80%8A%E7%94%9F%E6%B4%BB%E6%98%AF%E7%BE%8E%E5%A5%BD%E7%9A%84%E3%80%8B%E7%94%B5%E5%BD%B1%E5%89%A7%E6%9C%AC_1.2_%E7%BC%A9%E6%B7%B7_cut_26.198_38.406_C.npy"

# URLデコードしてオリジナルのファイル名を取得
decoded_path = urllib.parse.unquote(npy_url)
npy_filename = os.path.basename(decoded_path)

if not os.path.exists(npy_filename):
    print(f"NPYデータをダウンロード中...: {npy_filename}")
    !wget -O "$npy_filename" "$npy_url"
else:
    print(f"ファイルは既に存在します: {npy_filename}")

In [ ]:
# @title 3. ファイル名からの時間解析と動画の切り出し
# NPYファイル名から開始・終了時間を取得し、動画を加工します

# NPYファイルを探す
npy_files = glob.glob("*.npy")
if not npy_files:
    raise FileNotFoundError("NPYファイルが見つかりません")

target_npy = npy_files[0]
print(f"解析対象ファイル: {target_npy}")

# ファイル名から時間を抽出
try:
    parts = target_npy.split("cut_")[-1].split("_")
    start_time = float(parts[0])
    end_time = float(parts[1])
    duration = end_time - start_time
    print(f"開始時間: {start_time}秒")
    print(f"終了時間: {end_time}秒")
    print(f"記録秒数: {duration:.3f}秒")
except Exception as e:
    print(f"ファイル名の解析に失敗しました: {e}")
    duration = 10.627

# 動画の切り出し
output_video = "reference_video.mp4"
input_video = "Its-a-wonderful-life_1946.mp4"

# loudnorm (聞きやすい音量に揃える) を使用
cmd = [
    "ffmpeg", "-y",
    "-ss", "0",
    "-t", str(duration),
    "-i", input_video,
    "-af", "loudnorm=I=-16:TP=-1.5:LRA=11", # ラウドネス正規化
    "-c:v", "libx264",
    "-c:a", "aac",
    output_video
]

print("動画処理(音量正規化)を実行中...")
subprocess.run(cmd, check=True)
print(f"動画の作成完了: {output_video}")

In [ ]:
# @title 4. 映像に合わせて右から左へ移動
import numpy as np
import subprocess
import librosa
import soundfile as sf
from scipy.interpolate import interp1d
import glob

def create_binaural_audio_swap_and_invert(video_path, npy_path, output_wav):
    print(f"処理開始: {npy_path}")

    # 1. 動画からモノラル音声を抽出
    mono_wav = "temp_mono.wav"
    subprocess.run(["ffmpeg", "-y", "-i", video_path, "-ac", "1", "-ar", "48000", mono_wav], check=True)

    y, sr = librosa.load(mono_wav, sr=48000, mono=True)

    # 2. 位置情報の読み込み
    positions = np.load(npy_path)
    npy_fps = 20
    num_frames = positions.shape[0]

    # Y軸データを取得し、-1を掛けて動きを「反転」させる
    print("★ Y軸データを反転(-1倍)して、映像に合わせたX軸(右→左)として適用します")
    raw_target_axis = positions[:, 1] * -1.0

    # --- 動きの強調（正規化） ---
    min_val = np.min(raw_target_axis)
    max_val = np.max(raw_target_axis)

    print(f"反転後データ範囲: Min={min_val:.3f}, Max={max_val:.3f}")

    if max_val - min_val < 0.001:
        print("警告: 動きがほとんどありません。")
        x_coords = np.zeros_like(raw_target_axis)
    else:
        # Min-Max 正規化: データを 0.0 〜 1.0 に変換
        x_normalized = (raw_target_axis - min_val) / (max_val - min_val)

        # 範囲を -0.9 (左) 〜 0.9 (右) にマッピング
        x_coords = x_normalized * 1.8 - 0.9

    print(f"最終補正後(左右パン): Min={np.min(x_coords):.3f}, Max={np.max(x_coords):.3f}")

    # 時間軸の補間
    original_times = np.linspace(0, num_frames / npy_fps, num_frames)
    target_times = np.linspace(0, len(y) / sr, len(y))

    interpolator = interp1d(original_times, x_coords, kind='linear', fill_value="extrapolate")
    x_interpolated = interpolator(target_times)

    # 3. バイノーラル処理
    left_channel = np.zeros_like(y)
    right_channel = np.zeros_like(y)
    max_delay_samples = int(0.0007 * sr)

    for i in range(len(y)):
        x_pos = max(-1.0, min(1.0, x_interpolated[i]))

        # 線形パンニング
        norm_pos = (x_pos + 1) / 2
        vol_r = norm_pos
        vol_l = 1.0 - norm_pos

        # 遅延処理
        delay = int(x_pos * max_delay_samples)

        if delay > 0: # 右寄り
            idx_l = i + delay
            if idx_l < len(y): left_channel[idx_l] += y[i] * vol_l
            right_channel[i] += y[i] * vol_r
        else: # 左寄り
            idx_r = i - delay
            if idx_r < len(y): right_channel[idx_r] += y[i] * vol_r
            left_channel[i] += y[i] * vol_l

    # 音量正規化 (-2dBターゲット)
    max_amp = max(np.max(np.abs(left_channel)), np.max(np.abs(right_channel)))
    if max_amp > 0:
        target_peak = 0.8
        left_channel = (left_channel / max_amp) * target_peak
        right_channel = (right_channel / max_amp) * target_peak

    stereo_data = np.stack([left_channel, right_channel], axis=1)
    sf.write(output_wav, stereo_data, sr)
    print(f"バイノーラル音声生成完了: {output_wav}")

# 実行
npy_files = glob.glob("*.npy")
if npy_files:
    binaural_wav = "binaural_output.wav"
    create_binaural_audio_swap_and_invert("reference_video.mp4", npy_files[0], binaural_wav)
else:
    print("エラー: NPYファイルが見つかりません")

In [ ]:
# @title 5. 映像の結合と結果の比較再生
from IPython.display import Video, display, HTML
import subprocess
import os

final_output = "binaural_demo.mp4"
input_video = "reference_video.mp4"
input_audio = "binaural_output.wav"

# 結合処理
if os.path.exists(input_video) and os.path.exists(input_audio):
    print("映像と音声を結合中... (ラウドネス正規化: I=-16 LUFS)")
    cmd = [
        "ffmpeg", "-y",
        "-i", input_video,
        "-i", input_audio,
        "-c:v", "copy",       # 映像はそのまま
        # 【追加】ラウドネス正規化フィルタ
        # I=-16: 目標ラウドネス (Web/モバイル向け標準)
        # TP=-1.5: トゥルーピーク制限 (クリッピング防止)
        "-af", "loudnorm=I=-16:TP=-1.5:LRA=11",
        "-c:a", "aac",        # 音声エンコード
        "-map", "0:v:0",
        "-map", "1:a:0",
        "-shortest",
        final_output
    ]
    subprocess.run(cmd, check=True)
    print(f"結合完了: {final_output}")
else:
    print("エラー: 入力ファイルが見つかりません。Step 3, 4を実行してください。")

# 比較再生用関数
def display_video_result(file_path, title):
    if os.path.exists(file_path):
        display(HTML(f"<h4>{title}</h4>"))
        display(Video(file_path, embed=True, width=600))
    else:
        print(f"ファイルなし: {file_path}")

print("=== 🎧 実験結果の比較 (ヘッドホン推奨) ===")
print("※入力・出力ともに同じラウドネス基準(I=-16)で調整済みです。\n")

# Before: モノラル
display_video_result("reference_video.mp4", "1. Before: モノラル (Original)")

# After: 空間音響
display_video_result("binaural_demo.mp4", "2. After: 空間音響 (Binaural)")

In [ ]:
# @title 6. 【デバッグ】音源位置の可視化と数値確認
import matplotlib.pyplot as plt
import numpy as np
import glob

# 1. NPYファイルの読み込み
npy_files = glob.glob("*.npy")
if not npy_files:
    print("エラー: NPYファイルが見つかりません。Step 2を実行してください。")
else:
    target_npy = npy_files[0]
    print(f"解析対象: {target_npy}")

    positions = np.load(target_npy)
    # MRSAudioの仕様: (Frames, 3) -> [x, y, z] (または [x, y, z, ...])
    # 想定FPS = 20
    npy_fps = 20
    num_frames = positions.shape[0]
    duration_sec = num_frames / npy_fps

    raw_x = positions[:, 0]
    raw_y = positions[:, 1]

    timestamps = np.linspace(0, duration_sec, num_frames)

    print(f"総フレーム数: {num_frames}")
    print(f"データ長: {duration_sec:.2f}秒")
    print(f"X座標の範囲: Min={np.min(raw_x):.3f}, Max={np.max(raw_x):.3f}")

    # --- グラフ描画 ---
    plt.figure(figsize=(15, 5))

    # グラフ1: 時間 vs X座標（左右の動き）
    plt.subplot(1, 2, 1)
    plt.plot(timestamps, raw_x, label="X Position (Left-Right)", color="blue")
    plt.axhline(0, color='gray', linestyle='--') # 中央線
    plt.title("Time vs X-Position")
    plt.xlabel("Time (seconds)")
    plt.ylabel("X Coordinate (-1:Left, 1:Right)")
    plt.grid(True)
    plt.legend()

    # グラフ2: X-Y 平面軌跡（上から見た動き）
    plt.subplot(1, 2, 2)
    plt.plot(raw_x, raw_y, marker='o', markersize=2, linestyle='-', alpha=0.5, color="red")
    plt.title("Trajectory (Top View: X-Y)")
    plt.xlabel("X (Left-Right)")
    plt.ylabel("Y (Front-Back)")
    plt.grid(True)
    plt.axis('equal') # アスペクト比を合わせる

    # 開始点と終了点をマーク
    plt.plot(raw_x[0], raw_y[0], 'g^', markersize=10, label="Start")
    plt.plot(raw_x[-1], raw_y[-1], 'rs', markersize=10, label="End")
    plt.legend()

    plt.tight_layout()
    plt.show()

    # --- 数値デバッグ出力 (1秒ごと) ---
    print("\n=== 1秒ごとの座標データ (X) ===")
    print("Time(s) | X-Coord | Location")
    print("-" * 35)
    for t in range(int(duration_sec) + 1):
        frame_idx = min(int(t * npy_fps), num_frames - 1)
        x_val = raw_x[frame_idx]

        # 位置の目安
        loc_str = "Center"
        if x_val < -0.3: loc_str = "Left"
        elif x_val > 0.3: loc_str = "Right"

        print(f"{t:5.1f}s  | {x_val:6.3f}  | {loc_str}")